In [1]:
import os
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ConnectedAgentTool, MessageRole, ListSortOrder, ToolSet, FunctionTool
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv()
AIFOUNDRY_AGENT_PROJECT_NAME = os.getenv("AIFOUNDRY_AGENT_PROJECT_NAME")
AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME")


In [2]:
# Priority agent definition
priority_agent_name = "priority_agent"
priority_agent_instructions = """
Assess how urgent a ticket is based on its description.

Respond with one of the following levels:
- High: User-facing or blocking issues
- Medium: Time-sensitive but not breaking anything
- Low: Cosmetic or non-urgent tasks

Only output the urgency level and a very brief explanation.
"""

# Team agent definition
team_agent_name = "team_agent"
team_agent_instructions = """
Decide which team should own each ticket.

Choose from the following teams:
- Frontend
- Backend
- Infrastructure
- Marketing

Base your answer on the content of the ticket. Respond with the team name and a very brief explanation.
"""

# Effort agent definition
effort_agent_name = "effort_agent"
effort_agent_instructions = """
Estimate how much work each ticket will require.

Use the following scale:
- Small: Can be completed in a day
- Medium: 2-3 days of work
- Large: Multi-day or cross-team effort

Base your estimate on the complexity implied by the ticket. Respond with the effort level and a brief justification.
"""

# Instructions for the primary agent
triage_agent_instructions = """
 Triage the given ticket. Use the connected tools to determine the ticket's priority, 
 which team it should be assigned to, and how much effort it may take.
"""

# Connect to the agents client
agents_client = AgentsClient(
    endpoint=AIFOUNDRY_AGENT_PROJECT_NAME,
    credential=DefaultAzureCredential(
        exclude_environment_credential=True, 
        exclude_managed_identity_credential=True
    ),
)

# with agents_client:
# Create the priority agent on the Azure AI agent service
priority_agent = agents_client.create_agent(
    model=AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME,
    name=priority_agent_name,
    instructions=priority_agent_instructions
)

# Create a connected agent tool for the priority agent
priority_agent_tool = ConnectedAgentTool(
    id=priority_agent.id, 
    name=priority_agent_name, 
    description="Assess the priority of a ticket"
)


# Create the team agent and connected tool
team_agent = agents_client.create_agent(
    model=AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME,
    name=team_agent_name,
    instructions=team_agent_instructions
)
team_agent_tool = ConnectedAgentTool(
    id=team_agent.id, 
    name=team_agent_name, 
    description="Determines which team should take the ticket"
)


# Create the effort agent and connected tool
effort_agent = agents_client.create_agent(
    model=AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME,
    name=effort_agent_name,
    instructions=effort_agent_instructions
)
effort_agent_tool = ConnectedAgentTool(
    id=effort_agent.id, 
    name=effort_agent_name, 
    description="Determines the effort required to complete the ticket"
)


    # Create a main agent with the Connected Agent tools
agent = agents_client.create_agent(
    model=AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME,
    name="triage-agent",
    instructions=triage_agent_instructions,
    tools=[
        priority_agent_tool.definitions[0],
        team_agent_tool.definitions[0],
        effort_agent_tool.definitions[0]
    ]
)


# Create thread for the chat session
print("Creating agent thread.")
thread = agents_client.threads.create()
    



Creating agent thread.


In [ ]:
def send_prompt(agents_client, thread, prompt: str):

    # Send a pormpt to the agent
    message = agents_client.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=prompt,
    )
        
        
    # Create and process Agent run in thread with tools
    print("Processing agent thread. Please wait.")
    run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)


    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        if message.text_messages:
            last_msg = message.text_messages[-1]
            print(f"{message.role}:\n{last_msg.text.value}\n")
    



In [5]:
# Create the ticket prompt
prompt = "Users can't reset their password from the mobile app."
send_prompt(agents_client, thread, prompt)

Processing agent thread. Please wait.
user:
Users can't reset their password from the mobile app.

assistant:
Ticket Summary:
Users can't reset their password from the mobile app.

Triage Results:
- **Priority:** High – This is user-facing and blocking, preventing users from accessing their accounts .
- **Team Assignment:** Frontend team – The issue is likely with the mobile app's user interface or user experience, and possibly the code handling password reset functionality .
- **Estimated Effort:** Medium (2-3 days) – The fix will require investigation into frontend flows, potential API/backend integration, and thorough testing but is not expected to require a major overhaul unless deeper issues are discovered .

Sources:  
- Priority: connected_agent priority_agent  
- Team: connected_agent team_agent  
- Effort: connected_agent effort_agent

user:
Users can't reset their password from the mobile app.

assistant:
**Triage Summary**

- **Priority**: High  
  This is a user-facing, blo

In [6]:
#clean up agents

# Delete the agent when done
print("Cleaning up agents:")
agents_client.delete_agent(agent.id)
print("Deleted triage agent.")

# Delete the connected agents when done
agents_client.delete_agent(priority_agent.id)
print("Deleted priority agent.")
agents_client.delete_agent(team_agent.id)
print("Deleted team agent.")
agents_client.delete_agent(effort_agent.id)
print("Deleted effort agent.")

Cleaning up agents:
Deleted triage agent.
Deleted priority agent.
Deleted team agent.
Deleted effort agent.
